**Imports**

In [1]:
# Standard imports
import cv2
import numpy as np;
import os
import pandas as pd

**Blob detector to detect skyrmions**

In [2]:
# Blob detecotor parameters
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 10
params.maxThreshold = 200

# Filter by Area.
params.filterByArea = True
params.minArea = 10

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.87
    
# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01

# Different versions
ver = (cv2.__version__).split('.')
if int(ver[0]) < 3:
    detector = cv2.SimpleBlobDetector(params)
else: 
    detector = cv2.SimpleBlobDetector_create(params)

**Detect skyrmions**

In [3]:
directory = 'Rec_EDGE_300K_1L_50MA.out'

frames = []

no_skyrmions = 15

for filename in os.listdir(directory):
    # if png
    if filename.endswith(".png"):
        coordinates = None
        # Read image
        image = os.path.join(directory, filename)
        im = cv2.imread(image, cv2.IMREAD_GRAYSCALE)

        # Detect
        detector.empty()
        keypoints = detector.detect(im)
        
#         # Draw detected blobs as red circles.
#         # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
#         im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

#         # Show keypoints
#         cv2.imshow("Keypoints", im_with_keypoints)
#         cv2.waitKey(0)
        
        for p in keypoints:
            coordinates = np.append(coordinates, [p.pt[0], p.pt[1]]) if coordinates is not None else [p.pt[0], p.pt[1]]
            
        # if skyrmions arenot detected, then they are at the border of the image
        # impute putting them in the centre on the left side of the image
        while (len(coordinates) < 2 * no_skyrmions):
            coordinates = np.append(coordinates, [0, im.shape[:2][0] / 2]) if coordinates is not None else [0, im.shape[:2][0] / 2]
                               
        frames.append(list(coordinates))

**Save data to file**

In [4]:
# Save frames to file
with open('frames.txt', 'w') as f:
    for frame in frames:
        for i in range(0, len(frame), 2):
            f.write('(' + str(frame[i]) + ', ' + str(frame[i+1]) + '), ')
        f.write('\n\n')

**Place data in dataframe**

In [5]:
# data in columns ['frame', 'next_frame']
data = pd.DataFrame(columns=['frame', 'next_frame'])

for i in range(1, len(frames)):
    data = data.append({'frame': frames[i-1], 'next_frame': frames[i]}, ignore_index=True)

data

,frame,next_frame
0,"[71.79793548583984, 179.53582763671875, 30.651...","[83.91636657714844, 180.03187561035156, 121.75..."
1,"[83.91636657714844, 180.03187561035156, 121.75...","[94.73966217041016, 179.82919311523438, 56.100..."
2,"[94.73966217041016, 179.82919311523438, 56.100...","[130.58045959472656, 159.17333984375, 146.4812..."
3,"[130.58045959472656, 159.17333984375, 146.4812...","[115.47341918945312, 178.807861328125, 74.3923..."
4,"[115.47341918945312, 178.807861328125, 74.3923...","[130.07960510253906, 180.80287170410156, 83.26..."
...,...,...
794,"[759.0730590820312, 158.21380615234375, 696.59...","[766.320556640625, 157.91192626953125, 983.126..."
795,"[766.320556640625, 157.91192626953125, 983.126...","[992.1935424804688, 161.09410095214844, 777.61..."
796,"[992.1935424804688, 161.09410095214844, 777.61...","[572.3209228515625, 148.3568878173828, 912.637..."
797,"[572.3209228515625, 148.3568878173828, 912.637...","[797.6422119140625, 157.04588317871094, 743.31..."


**Split data for training and testing**

In [6]:
from sklearn.model_selection import train_test_split

X = data['frame'].tolist()
y = data['next_frame'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

**Train model with Linear Regression**

In [7]:
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression()

**Model evaluation**

In [8]:
from sklearn.metrics import mean_squared_error, r2_score

# model evaluation for training set
y_train_predict = lin_model.predict(X_train)
rmse = (np.sqrt(mean_squared_error(y_train, y_train_predict)))
r2 = r2_score(y_train, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
y_test_predict = lin_model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, y_test_predict)))
r2 = r2_score(y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for training set
--------------------------------------
RMSE is 152.1772808486889
R2 score is 0.27242325152731145


The model performance for testing set
--------------------------------------
RMSE is 151.7964368378906
R2 score is 0.1922438437784907


**Train with Polynoial Regression**

In [9]:
from sklearn.preprocessing import PolynomialFeatures

def create_polynomial_regression_model(degree):
    poly_features = PolynomialFeatures(degree=degree)

    # transform the features to higher degree features.
    X_train_poly = poly_features.fit_transform(X_train)

    # fit the transformed features to Linear Regression
    poly_model = LinearRegression()
    poly_model.fit(X_train_poly, y_train)

    # predicting on training data-set
    y_train_predicted = poly_model.predict(X_train_poly)

    # predicting on test data-set
    y_test_predict = poly_model.predict(poly_features.fit_transform(X_test))

    # evaluating the model on training dataset
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_predicted))
    r2_train = r2_score(y_train, y_train_predicted)

    # evaluating the model on test dataset
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))
    r2_test = r2_score(y_test, y_test_predict)

    print("The model performance for the training set")
    print("-------------------------------------------")
    print("RMSE of training set is {}".format(rmse_train))
    print("R2 score of training set is {}".format(r2_train))

    print("\n")

    print("The model performance for the test set")
    print("-------------------------------------------")
    print("RMSE of test set is {}".format(rmse_test))
    print("R2 score of test set is {}".format(r2_test))

In [10]:
create_polynomial_regression_model(2)

The model performance for the training set
-------------------------------------------
RMSE of training set is 46.407560418823216
R2 score of training set is 0.8696725099837165


The model performance for the test set
-------------------------------------------
RMSE of test set is 377.34935096324915
R2 score of test set is -5.540682774708514
